### Μεταφόρτωση αρχείων από το τοπικό σύστημα.

Με την βιβλιοθήκη <code>files</code> από το <code>google.colab</code> ανεβάζω τα απαραίτητα αρχεία για την υλοποίηση της άσκησης.

In [63]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving dataset-HAR-PUC-Rio.csv to dataset-HAR-PUC-Rio.csv
User uploaded file "dataset-HAR-PUC-Rio.csv" with length 14283211 bytes


# Data Preprocessing
> **Γιατί κάνουμε data preprocessing**: Το data set μας πολύ πιθανών να αποτελείται απο features διαφορετικών range.Για παράδειγμα, το ένα feature μπορεί να είναι η ηλικία ενός ατόμου που κυμαίνεται στο διάστημα $[0,120]$, ενώ ένα άλλο feature μπορεί να αναφέρεται σε κάποιο χρηματικό ποσό που κυμαίνεται στο διάστημα $[0,100000]$.Αυτό έχει ως αποτέλεσμα το δεύτερο feature να επηρεάζει περισσότερο την έξοδο από το πρώτο.Επομένως προσθέτουμε ένα bias ως προς το δεύτερο χαρακτηριστικό.<br><br>
>**Normalization**: Με αυτή την μέθοδο μεταφέρουμε το εύρος κάθε feature στο διάστημα $[0,1]$.Έτσι επιλύουμε το παραπάνω πρόβλημα.Αυτή η μέθοδος χρησιμοποιείται σε αλγόριθμους που δεν κάνουν υποθέσεις για την κατανομή των δεδομένων , όπως τα **neural networks**.Για αυτό τον λόγο θα χρησιμοποιηθεί αυτή η τεχνική για το πρόβλημά μας.<br><br>
>**Standarization**: Αυτή η μέθοδος υποθέτει ότι τα δεδομένα ακολουθούν gaussian distribution μετασχηματίζοντας τα έτσι ώστε να έχουν μηδενική μέση τιμή και διασπορά ίση με 1.Χρησιμοποιείται λοιπόν σε αλγόριθμους που υποθέτουν ότι τα δεδομένα ακολουθούν gaussian distribution όπως το **linear regresion**.<br><br>
>**Centralization**: Αυτή η μέθοδος μετασχηματίζει τα δεδομένα έτσι ώστε να έχουν μηδενική μέση τιμή. Παρόλα αυτά δεν λύνει το πρόβλημα που αναφέρθηκε παραπάνω.

In [121]:
from pandas.tseries.offsets import YearBegin
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

# read dataset
dataset = pd.read_csv("dataset-HAR-PUC-Rio.csv",delimiter=";", low_memory=False)

# create dictionary that maps strings to numbers
string_to_num = {'debora': 1, 'katia': 2, 'wallace': 3, 'jose_carlos': 4, 'Woman': 1, 'Man': 2 }

# lambda function
def map_strings_to_nums(col):
    return col.apply(lambda x: string_to_num[x] if x in string_to_num else x)

# apply lambda function to string columns
dataset[['user', 'gender', 'class']] = dataset[['user', 'gender', 'class']].apply(map_strings_to_nums)

# replace ',' with '.' so astype can work
dataset['how_tall_in_meters'] = dataset['how_tall_in_meters'].str.replace(',','.')
dataset['body_mass_index'] = dataset['body_mass_index'].str.replace(',','.')

# split features and Output
X = dataset.drop('class', axis=1)

labels = dataset['class'].values.reshape(-1, 1)
encoder = OneHotEncoder()
Y = pd.DataFrame(encoder.fit_transform(labels).toarray())

X = X.astype(float)
Y = Y.astype(float)
# Normalize features
#scaler = MinMaxScaler()
#dataset_normalized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)


# Cross Validation

> **Γιατί;**: Αυτή η μέθοδος χρησιμοποιείται για να εξασφαλίσουμε ότι το score του αλγορίθμου μας δεν εξαρτάται από τον τρόπο που επιλέγουμε τα <code>train_set</code> και <code>test_set</code>.<br><br>
> **Πως**;: Χωρίζουμε το dataset σε Κ folds και χρησιμοποιούμε τα Κ-1 για <code>train_set</code> και το υπολοιπώμενο για <code>test_set</code> μέχρι και τα K folds να έχουν γίνει <code>test_set</code>.Στο πρόβλημα μας θα χρησιμοποιήσουμε 5-fold cross validation.

In [122]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=39)

for fold, (train_idx, test_idx) in enumerate(kf.split(X, Y)):
    print(f"Fold {fold+1} Train Index: {train_idx}")
    print(f"Fold {fold+1} Test Index: {test_idx}")

Fold 1 Train Index: [     1      2      3 ... 165628 165630 165631]
Fold 1 Test Index: [     0      7     11 ... 165616 165626 165629]
Fold 2 Train Index: [     0      1      3 ... 165629 165630 165631]
Fold 2 Test Index: [     2     12     20 ... 165608 165619 165628]
Fold 3 Train Index: [     0      1      2 ... 165628 165629 165630]
Fold 3 Test Index: [     6     10     15 ... 165617 165627 165631]
Fold 4 Train Index: [     0      1      2 ... 165629 165630 165631]
Fold 4 Test Index: [     4     16     18 ... 165622 165623 165624]
Fold 5 Train Index: [     0      2      4 ... 165628 165629 165631]
Fold 5 Test Index: [     1      3      5 ... 165620 165625 165630]


# Επιλογή Αρχιτεκτονικής

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
import numpy as np


metrics = []

for train_idx, test_idx in kf.split(dataset):
    #number of input features
    input_dim = 18
    #number of nodes in hidden layer
    hidden_dim = 64
    # create a sequential model
    model = Sequential()
    # add a hidden layer with hidden_dim neurons and ReLU activation function
    model.add(Dense(hidden_dim, input_dim=input_dim, activation='relu'))
    # add an output layer with 5 neurons and softmax activation function
    model.add(Dense(5, activation='softmax'))
    #compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = Y.iloc[train_idx], Y.iloc[test_idx]

    history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
    metrics.append(model.evaluate(X_test, y_test))

avg_metrics = np.mean(metrics, axis=0)
print(f'Average loss: {avg_metrics[0]}, Average accuracy: {avg_metrics[1]}')


Epoch 1/10
4141/4141 [==============================] - 11s 2ms/step - loss: 1.0765 - accuracy: 0.8563 - val_loss: 0.3969 - val_accuracy: 0.9049
Epoch 2/10
4141/4141 [==============================] - 10s 2ms/step - loss: 0.3271 - accuracy: 0.9203 - val_loss: 0.3581 - val_accuracy: 0.9321
Epoch 3/10
4141/4141 [==============================] - 9s 2ms/step - loss: 0.2595 - accuracy: 0.9371 - val_loss: 0.2058 - val_accuracy: 0.9477
Epoch 4/10
4141/4141 [==============================] - 10s 2ms/step - loss: 0.2201 - accuracy: 0.9456 - val_loss: 0.1598 - val_accuracy: 0.9629
Epoch 5/10
4141/4141 [==============================] - 10s 2ms/step - loss: 0.1768 - accuracy: 0.9541 - val_loss: 0.1851 - val_accuracy: 0.9627
Epoch 6/10
4141/4141 [==============================] - 11s 3ms/step - loss: 0.1595 - accuracy: 0.9591 - val_loss: 0.1652 - val_accuracy: 0.9532
Epoch 7/10
4141/4141 [==============================] - 15s 4ms/step - loss: 0.1419 - accuracy: 0.9626 - val_loss: 0.1219 - val_acc